In [2]:
from constraint import *
import re as re

b'%%% Vector of UAS model names (9) which will need for pretty printing\r\nAR200\r\nDisco\r\nFalcon\r\nMavic\r\nM600\r\nP4\r\nPHK\r\nScanEagle\r\nSolo\r\n%%%%\r\nMT A 110111001  % SA\r\nMT B 101111101 % Inspection\r\nMT C 110111101 % ground search\r\nMT D 010111001  % water search\r\nMT E 101111111  % debris estimation\r\nMT F 110111001 % tactical SA\r\nMT G 000000000 % delivery\r\n%%% Pilot name, UAS models can fly, favorite UAS model\r\nP Merrick 010101001 000100000\r\nP Adams 110111101 000100000\r\nP Gingrich 000101101 000100000\r\nP Major 000000010 000000010\r\n%%% Number of UAS available by model\r\nNUAS 111412511  % 9 characters\x97each is the number of a UAS model, no more than 9 of any model\r\n%%% Mission name and Mission Type\r\nM 1 E \r\nM 2 E\r\nM 3 E\r\nM 4 E\r\nM 5 B\r\nM 6 E\r\nM 7 B\r\nM 8 B\r\nM 9 E\r\nM 10 E\r\nM 11 E\r\nM 12 E\r\nM 13 E\r\nM 14 B\r\nM 15 B\r\nM 16 B\r\nM 17 B\r\nM 18 F\r\nM 19 F\r\nM 20 A'

In [52]:
def parse_inputs(path):
    models = []
    mission_types = {}
    pilots = {}
    missions = {}
    nuas = []
    f = open(path, encoding="utf8", errors='ignore')
    inputs = f.readlines()
    f.close()
    for i in range(len(inputs)):
        line = inputs[i]
        if not line[0] == "%":
            line = line.rstrip()
            if line[0:3] == "MT ":
                line = line.replace("  "," ")
                line = line.split(" ")  
                mission_type = line[1]
                compat_string = list(line[2])
                compat = [int(i) for i in compat_string]            
                mission_types[mission_type] = compat
            elif line[0:2] == "P ":
                line = line.replace("  "," ")
                line = line.split(" ")
                pilot = line[1]
                compat_string = list(line[3])
                compat = [int(i) for i in compat_string]            
                pilots[pilot] = compat
            elif line[0:2] == "M ":
                line = line.replace("  "," ")
                line = line.replace("M ", "")
                line = line.replace("M", "")
                line = line.split(" ")
                mission_num = line[0]
                mission_type = line[1][0]
                compat = mission_types[mission_type]
                missions[mission_num] = compat
            elif "NUAS" in line:
                line = line.replace("  "," ")
                line = line.split(" ")
                l_nuas = list(line[1])
                l_nuas = [int(i) for i in l_nuas]
                for i in l_nuas:
                    nuas.append(i)
            else:   
                models.append(line)
#     print()
    print("Number of missions:", len(missions.keys()), "and number of mission types: ", len(list(mission_types.keys())))
    print("Number of pilots:", len(pilots.keys()))
    print("Number of unique models:", len(models), "and number of total models:", sum(nuas))
    return(models, pilots, missions, nuas, mission_types)

    
models, pilots, missions, nuas, mission_types = parse_inputs("sample2.txt")   

Number of missions: 20 and number of mission types:  7
Number of pilots: 4
Number of unique models: 9 and number of total models: 17


In [49]:
# number of missions
n_missions = len(missions.keys())

# number of pilots
n_pilots = len(pilots.keys())

#number of unique models
n_unique_models = len(models)

In [14]:
problem = Problem()

pilot_model_variables = []
for model in range(n_unique_models):
    for pilot in range(n_pilots):
        var = "p"+str(pilot)+"_m"+str(model)+"_"
        problem.addVariable(var, [0,1])
        pilot_model_variables.append(var)

mission_model_variables = []
for type in range(n_missions):
    for model in range(n_unique_models):
        var = "t"+str(type)+"_m"+str(model)+"_"
        problem.addVariable(var, [0,1])
        mission_model_variables.append(var)




# add constraints

#each pilot drives only uav:
for pilot in range(n_pilots):
    current_pilot = "p"+str(pilot)+"_"
    current_pilot_vars = [s for s in pilot_model_variables if current_pilot in s]
    problem.addConstraint(MaxSumConstraint(1), current_pilot_vars)

#pilot model compatibility constraint
pilot_keys = list(pilots.keys())
for pilot in range(n_pilots):
    compatible_models = pilots[pilot_keys[pilot]]
    for model in range(n_unique_models):
        var = "p"+str(pilot)+"_m"+str(model)+"_"
        compatibility = compatible_models[model]
        problem.addConstraint(lambda x: x <= compatibility, [var])
        
    
#Number of models constraint?
for model in range(n_unique_models):
    current_model = "m"+str(model)+"_"
    current_model_vars = [s for s in pilot_model_variables if current_model in s]
    num_available = nuas[model]
    if len(current_model_vars) > 0:
        problem.addConstraint(MaxSumConstraint(num_available), current_model_vars)
        
# one mission one UAV
for type in range(n_missions):
    current_mission = "t"+str(type)+"_"
    current_mission_vars = [s for s in mission_model_variables if current_mission in s]
    problem.addConstraint(ExactSumConstraint(1), current_mission_vars)


#mission model compatibility constaint
mission_keys = list(missions.keys())
for type in range(n_missions):
    compatible_models = missions[mission_keys[type]]
    for model in range(n_unique_models):
        var = "t"+str(type)+"_m"+str(model)+"_"
        compatibility = compatible_models[model]
        problem.addConstraint(lambda x: x <= compatibility, [var])
        
# each uav model can be used for a maximum of 3 times as many as the number of pilots driving that model
# and each uav model must be used at least 1 time as many as the number of pilots driving that model
for model in range(n_unique_models):
    var = "m"+str(model)+"_"
    num_models = nuas[model]
    current_model_missions = [s for s in mission_model_variables if var in s]
    current_model_pilots = [s for s in pilot_model_variables if var in s]
    problem.addConstraint(MaxSumConstraint(3), current_model_missions)
    problem.addConstraint(MinSumConstraint(1), current_model_missions)
#     problem.addConstraint(lambda m, p: sum(p) <= sum(m), (current_model_missions, current_model_pilots))
#     problem.addConstraint(lambda m, p: 3*sum(p) >= sum(m), (current_model_missions, current_model_pilots))


solution = problem.getSolution()

In [ ]:
g

In [15]:
print(nuas)
sol_keys = list(solution.keys())
keys = [key for key in sol_keys if "m4_" in key ]
for key in keys:
    print(key+":"+str(solution[key]))

[1, 1, 1, 4, 1, 2, 5, 1, 1]
t0_m4_:0
t10_m4_:0
t11_m4_:0
t12_m4_:0
t13_m4_:0
t14_m4_:0
t15_m4_:0
t16_m4_:0
t17_m4_:0
t18_m4_:0
t19_m4_:0
t1_m4_:0
t2_m4_:1
t3_m4_:1
t4_m4_:1
t5_m4_:0
t6_m4_:0
t7_m4_:0
t8_m4_:0
t9_m4_:0
p0_m4_:0
p1_m4_:0
p2_m4_:0
p3_m4_:0


In [16]:
mission_keys = list(missions.keys())
# solution_keys = list(solution.keys())
for mission in range(n_missions):
    current_mission = None
    current_model = None
    current_pilot = None
    current_mission = mission_keys[mission]
    
    current_mission_model_variables = [i for i in mission_model_variables if "t"+str(mission)+"_m" in i]
    for var in current_mission_model_variables:
        if solution[var] == 1:
            model_num = re.findall(r'\d+', var)[1]
            current_model = model_num + "_" + models[int(model_num)]
            current_pilot_model_variables = [i for i in pilot_model_variables if "m"+model_num+"_" in i]
#             print(current_pilot_model_variables)
            for pilot_var in current_pilot_model_variables:
#                 print(pilot_var, solution[pilot_var])
                if solution[pilot_var] == 1:
                    current_pilot_num = re.findall(r'\d+', pilot_var)[0]
                    current_pilot = list(pilots.keys())[int(current_pilot_num)]
    print("Mission:", current_mission)
    print("Model:", current_model)
    print("Pilot:", current_pilot)
    print("\n")

        

Mission: 1
Model: 0_AR200
Pilot: Merrick


Mission: 2
Model: 3_Mavic
Pilot: Major


Mission: 3
Model: 4_M600
Pilot: None


Mission: 4
Model: 4_M600
Pilot: None


Mission: 5
Model: 4_M600
Pilot: None


Mission: 6
Model: 5_P4
Pilot: None


Mission: 7
Model: 5_P4
Pilot: None


Mission: 8
Model: 6_PHK
Pilot: None


Mission: 9
Model: 7_ScanEagle
Pilot: None


Mission: 10
Model: 8_Solo
Pilot: None


Mission: 11
Model: 0_AR200
Pilot: Merrick


Mission: 12
Model: 0_AR200
Pilot: Merrick


Mission: 13
Model: 1_Disco
Pilot: Adams


Mission: 14
Model: 1_Disco
Pilot: Adams


Mission: 15
Model: 1_Disco
Pilot: Adams


Mission: 16
Model: 2_Falcon
Pilot: Gingrich


Mission: 17
Model: 2_Falcon
Pilot: Gingrich


Mission: 18
Model: 2_Falcon
Pilot: Gingrich


Mission: 19
Model: 3_Mavic
Pilot: Major


Mission: 20
Model: 3_Mavic
Pilot: Major




In [17]:
pilots
problem.getSolutions()

KeyboardInterrupt: 